In [26]:
import pandas as pd
import numpy as np
import random
import math

import import_ipynb
# from decision_trees import *

In [27]:
class node:

    def __init__(self, left=None, right=None, threshold=None, feature=None, g_score=None, classif = None):
        self.left = left
        self.right = right
        self.threshold = threshold
        self.feature = feature
        self.g_score = g_score
        self.classif = classif


In [28]:
class decision_tree:

    """
    Limitation : only designed to handle classification with 0 and 1 as classes.
    """

    def __init__(self,max_depth,min_sample_size):
        self.root = None

        self.max_depth = max_depth
        self.min_sample_size = min_sample_size

    def make_split(self, X, feature, threshold, Y):
        """ 
        desc : makes two lists from the data

        X : (numpy) numpy data to be split
        feature : (int) the feature number on which we will split the data
        threshold : (int) the threshold for splitting
        Y : (numpy) the series storing classification of each datapoint

        returns : (tuple of lists) returns two lists having the classifications of split data in list format
        """
        l = []
        g = []
        for i in range(X.shape[0]):
            if X[i][feature]<=threshold:
                l.append(Y[i])
            else:
                g.append(Y[i])
        
        return (l,g)

    def gini(self,l):
        """
        desc: calculates gini score

        l : (list) stores the classification of each datapoint

        returns : (tuple) return gini score(int) and the most occuring class in l
        """
        gin = 0
        p = 0
        r = (l.count(0)/len(l),l.count(1)/len(l))

        classes = np.unique(l)

        for i in classes:
            t = l.count(i)
            if len(l)==0:
                t = 0
            else:
                t = t/len(l)
            gin += t*(1-t)
            if t>p : 
                p = t
        
        return (-gin,r)

    def split_data(self,X,feature,threshold):
        """
        desc : splits the dataframe into two dataframes

        X : (numpy) numpy data to be split
        feature : (int) the feature number on which we will split the data
        threshold : (int) the threshold for splitting
        
        returns : (tuple) two split dataframes
        """

        l = np.zeros((1,X.shape[1]))
        g = np.zeros((1,X.shape[1]))

        for i in range(X.shape[0]):

            if X[i][feature] <= threshold:
                l = np.vstack([l, X[i]])
            else:
                g = np.vstack([g, X[i]])

        return (l[1:],g[1:])

    def best_split(self,X,Y):
        """
        desc : finds the best split for the data

        X : (numpy) numpy data for which we need the best split
        Y: (numpy) the series storing classification of each datapoint

        returns : (tuple) returns the (feature(best split feature),threshold(best split feature),gini_score(data),classification(most occuring class of data)) of the best data
            return gin as -inf if no such split exists
        """

        gin = float('-inf')
        f = ""
        thresh = float('-inf')

        for i in range(X.shape[1]):
            for j in range(X.shape[0]):
                l,g = self.make_split(X,i,X[j][i],Y)
                if len(l)<self.min_sample_size or len(g)<self.min_sample_size:
                    continue
                num = (len(l)*self.gini(l)[0] + len(g)*self.gini(g)[0])/(len(l)+len(g))
                if num>gin:
                    gin = num
                    f = i
                    thresh = X[j][i]

        t = self.gini(list(Y))
        if thresh == float('-inf') and f == "":
            return (f,thresh,t[0],-1)
        l,g = self.make_split(X,f,thresh,Y)
        t_l = self.gini(l)
        t_g = self.gini(g)
        return (f,thresh,t[0],(t_l[1],t_g[1]))

    def build_tree(self,X,depth,Y):
        """
        desc : builds our decision tree

        X : (numpy) current numpy data of node
        depth : (int) the current depth of the tree
        Y: (numpy) the series storing classification of each datapoint

        return : (no return type)
        """

        tup = self.best_split(X,Y)
        a = node(threshold=tup[1],feature=tup[0],g_score=tup[2],classif=tup[3])

        #if size constraint is violated
        if tup[1] == float('-inf') and tup[0] == "":
            return

        if depth == 0:
            self.root = a
            l,g = self.split_data(X,tup[0],tup[1])
            Y_l,Y_g = self.make_split(X,tup[0],tup[1],Y)
            self.root.left = self.build_tree(l,depth+1,Y_l)
            self.root.right = self.build_tree(g,depth+1,Y_g)
            return self.root
            
        elif depth<=self.max_depth and depth>0:
            n = a
            l,g = self.split_data(X,tup[0],tup[1])
            Y_l,Y_g = self.make_split(X,tup[0],tup[1],Y)
            n.left = self.build_tree(l,depth+1,Y_l)
            n.right = self.build_tree(g,depth+1,Y_g)
            return n

    def print_tree(self,n,depth):
        if n!=None :
            s = '   '*depth
            print(s,n.threshold,n.feature,n.classif)
            self.print_tree(n.left,depth+1)
            self.print_tree(n.right,depth+1)

    def predict_row(self,X):
        """
        desc : predicts the output for a single row of features from the dataset

        X : (numpy row) row of features

        return : (int) the appropriate classification
        """
        r = self.root
        l = True

        while l:
            if X[r.feature]<=r.threshold:
                if r.left == None:
                    l = True
                    break
                r = r.left
            else:
                if r.right == None:
                    l = False
                    break
                r = r.right
        
        c = (-1,-1)

        if l:
            c = r.classif[0]
        else:
            c = r.classif[1]

        if c[0]>=c[1]:
            return 0
        else:
            return 1


    def predict(self,X_test):
        """
        desc : predicts the output for a the test dataset

        X : (numpy) the test dataset

        return : (list) the appropriate classifications of each row
        """
        l = []
        for i in range(X_test.shape[0]):
            l.append(self.predict_row(X_test[i]))
        return l


In [29]:
class random_tree:

    def __init__(self):
        """
        desc : creates an object which holds the random decision tree and the mapping for the features of random dec. tree to our dataset features

        return : (None)
        """
        self.mapping = None
        self.tree = None

In [30]:
class random_forest:

    def __init__(self, num_trees , min_sample_size , max_depth ):
        """
        desc : creates an object which stores relevant parameters for random forest building

        num_trees : (int) number of decision trees our random forest consists of
        min_sample_size : (int) minimum number of rows in a node
        max_depth : (int) the maximum depth a random tree can attain

        return : (None)
        """
        self.num_trees = num_trees
        self.min_sample_size = min_sample_size
        self.max_depth = max_depth
        self.list_trees = None

    def build_random_tree(self,X,Y):
        """
        desc : builds one random tree in our random forest
        
        X : (numpy array) the dataset for which we want to make a random forest
        Y : (numpy) the results of dataset for which we want random forest

        return : (object of random_tree class) object storing the mapping of features and the decision tree made on bootstrapped data
        """
        o = random_forest(num_trees=self.num_trees,min_sample_size=self.min_sample_size,max_depth=self.max_depth)
        n = math.floor(np.sqrt(X.shape[1]))
        l = list(range(0,n))
        m = {}
        for i in l:
            m[i] = random.randint(0,X.shape[1]-1)
        o.mapping = m
        
        X_rand = np.zeros((X.shape[0],n))
        Y_rand = []

        for i in range(X.shape[0]):
            r = random.randint(0,X.shape[0]-1)
            for j in range(n):
                X_rand[i][j] = X[r][m[j]]
                Y_rand.append(Y[r])

        dec = decision_tree(max_depth=self.max_depth,min_sample_size=self.min_sample_size)
        dec.build_tree(X_rand,0,Y_rand)
        o.tree = dec
        return o

    def build_random_forest(self,X,Y):
        """
        desc : builds random forest in a list format
        
        X : (numpy array) the dataset for which we want to make a random forest
        Y : (numpy) the results of dataset for which we want random forest

        return : (list of object of random_tree class) returns a list of objects having the decision tree and the appropriate feature mapping to original dataset
        """
        l = []
        for i in range(self.num_trees):
            o = self.build_random_tree(X,Y)
            l.append(o)
        self.list_trees = l

    def predict_r(self,X):
        """
        desc : predicts the classification of the test dataset
        
        X : (numpy) the test dataset value for which we want to predict output

        return : (int) returns the classification of the datapoint
        """
        res = []
        l = self.list_trees
        for i in range(len(l)):
            m = l[i].mapping
            X_new = np.zeros((1,len(m)))
            for j in range(len(m)):
                t = m[j]
                X_new[0][j] = X[t]
            
            dec = l[i].tree
            d = dec.root
            res.append(dec.predict_row(X_new.tolist()[0]))
        agg = sum(res)/len(res)
        if agg>0.5:
            return 1
        else:
            return 0

    def predict(self,X):
        """
        desc : predicts the classification of the test dataset
        
        X : (numpy array) the test dataset for which we want to predict output

        return : (list of int) returns the classification of each datapoint in our test dataset
        """

        res = []
        for i in range(X.shape[0]):
            res.append(self.predict_r(X[i]))
        return res

    def print_forest(self):
        l = self.list_trees
        for i in range(len(l)):
            print("Forest building completed")

In [31]:
#converting input to the required format

from sklearn.impute import SimpleImputer

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

relevant_features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']

imputer = SimpleImputer(strategy='most_frequent')
train[relevant_features] = imputer.fit_transform(train[relevant_features])
test[relevant_features] = imputer.transform(test[relevant_features])

#encoding
train['Sex'] = train['Sex'].map({'male':0, 'female':1})
test['Sex'] = test['Sex'].map({'male':0, 'female':1})
train['Embarked'] = train['Embarked'].map({'S':0,'C':1,'Q':2})
test['Embarked'] = test['Embarked'].map({'S':0,'C':1,'Q':2})

X = train[relevant_features].to_numpy()
X_test = test[relevant_features].to_numpy()
Y = train['Survived'].to_numpy()

In [32]:
o = random_forest(num_trees = 2,min_sample_size=10,max_depth=5)

o.build_random_forest(X,Y)
o.print_forest()

Forest building completed
Forest building completed


In [33]:
test['Survived'] = o.predict(X_test)
submissions = test[['PassengerId', 'Survived']]
print(submissions)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         0
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [34]:
print(submissions['Survived'].sum())

0
